# About this notebook

This notebook integrates previously acquired gridMet data for San Diego county with historical fire perimeters. It creates two new features for the gridMet dataframe that indicate (1) whether a fire occurred at the location and date specified by the record and (2) the acres burned by the fire if so. 

In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
%%time
df = pd.read_parquet('gridMet.parquet.gz').reset_index()
df = df[~(df.date.isna() | df.precipitation_amount_mm.isna())]
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
df.drop(['latitude', 'longitude'], axis=1, inplace=True)

Wall time: 59.8 s


In [3]:
%%time
# Expand the latitude and longitude from a point to a rectangle.
# The extent of the rectangle is 1/24th of a degree in both directions,
# which is the displacement between points of the original gridMet data.
from shapely.geometry import Polygon
dx, dy = 1/24, 1/24
def to_box(p):
    poly = Polygon([(p.x,      p.y),
                    (p.x + dx, p.y),
                    (p.x + dx, p.y + dy),
                    (p.x,      p.y + dy),
                   ])
    return poly
df = df.set_geometry(df.geometry.apply(to_box))
df.head()

Wall time: 10min 8s


,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry
0,1999-01-01,0.0,40.3,0.00589,138.0,123.0,1.6,293.1,281.1,24.0,16.0,15.5,34.0,1.7,0.74,"POLYGON ((-117.97500 33.56667, -117.93333 33.5..."
1,1999-01-01,0.0,39.8,0.00590,137.2,123.0,1.6,293.1,281.2,24.0,16.0,15.5,35.0,1.7,0.74,"POLYGON ((-117.93333 33.56667, -117.89167 33.5..."
2,1999-01-01,0.0,38.2,0.00580,137.2,123.0,1.7,293.2,281.2,26.0,15.2,14.9,37.0,1.7,0.77,"POLYGON ((-117.89167 33.56667, -117.85000 33.5..."
3,1999-01-01,0.0,36.4,0.00567,137.3,49.0,1.8,293.3,280.3,27.0,15.0,14.8,38.0,1.8,0.76,"POLYGON ((-117.85000 33.56667, -117.80833 33.5..."
4,1999-01-01,0.0,33.8,0.00538,137.3,49.0,1.9,293.2,279.9,29.0,13.3,13.6,43.0,1.9,0.80,"POLYGON ((-117.80833 33.56667, -117.76667 33.5..."


In [4]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6826300 entries, 0 to 10676520
Data columns (total 16 columns):
 #   Column                                            Dtype         
---  ------                                            -----         
 0   date                                              datetime64[ns]
 1   precipitation_amount_mm                           float64       
 2   relative_humidity_%                               float64       
 3   specific_humidity_kg/kg                           float64       
 4   surface_downwelling_shortwave_flux_in_air_W m-2   float64       
 5   wind_from_direction_Degrees Clockwise from north  float64       
 6   wind_speed_m/s                                    float64       
 7   max_air_temperature_K                             float64       
 8   min_air_temperature_K                             float64       
 9   burning_index_g_Unitless                          float64       
 10  dead_fuel_moisture_100hr_Percent 

In [5]:
df.describe()

,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa
count,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06
mean,8.304122e-01,3.000426e+01,6.503330e-03,2.385361e+02,2.266884e+02,2.980093e+00,2.976084e+02,2.834156e+02,3.714322e+01,1.201792e+01,1.275536e+01,5.026444e+01,4.325072e+00,1.354994e+00
std,4.286879e+00,1.769667e+01,2.908315e-03,7.989067e+01,7.785217e+01,1.255808e+00,7.771557e+00,6.103147e+00,1.910992e+01,4.536604e+00,4.035236e+00,2.096610e+01,1.928999e+00,9.730105e-01
min,0.000000e+00,1.000000e-01,2.400000e-04,4.500000e+00,0.000000e+00,4.000000e-01,2.686000e+02,2.575000e+02,0.000000e+00,1.900000e+00,3.400000e+00,0.000000e+00,2.000000e-01,0.000000e+00
25%,0.000000e+00,1.540000e+01,4.310000e-03,1.649000e+02,1.970000e+02,2.200000e+00,2.921000e+02,2.791000e+02,2.700000e+01,8.100000e+00,9.600000e+00,3.400000e+01,2.700000e+00,6.300000e-01
50%,0.000000e+00,2.670000e+01,6.200000e-03,2.456000e+02,2.410000e+02,2.700000e+00,2.970000e+02,2.833000e+02,3.700000e+01,1.210000e+01,1.290000e+01,4.700000e+01,4.200000e+00,1.090000e+00
75%,0.000000e+00,4.260000e+01,8.430000e-03,3.129000e+02,2.770000e+02,3.500000e+00,3.031000e+02,2.879000e+02,5.000000e+01,1.540000e+01,1.560000e+01,6.600000e+01,5.700000e+00,1.830000e+00
max,2.871000e+02,1.000000e+02,2.392000e-02,3.777000e+02,3.600000e+02,1.620000e+01,3.227000e+02,3.074000e+02,1.510000e+02,3.060000e+01,3.080000e+01,1.100000e+02,1.320000e+01,7.170000e+00


In [6]:
df.head()

,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry
0,1999-01-01,0.0,40.3,0.00589,138.0,123.0,1.6,293.1,281.1,24.0,16.0,15.5,34.0,1.7,0.74,"POLYGON ((-117.97500 33.56667, -117.93333 33.5..."
1,1999-01-01,0.0,39.8,0.00590,137.2,123.0,1.6,293.1,281.2,24.0,16.0,15.5,35.0,1.7,0.74,"POLYGON ((-117.93333 33.56667, -117.89167 33.5..."
2,1999-01-01,0.0,38.2,0.00580,137.2,123.0,1.7,293.2,281.2,26.0,15.2,14.9,37.0,1.7,0.77,"POLYGON ((-117.89167 33.56667, -117.85000 33.5..."
3,1999-01-01,0.0,36.4,0.00567,137.3,49.0,1.8,293.3,280.3,27.0,15.0,14.8,38.0,1.8,0.76,"POLYGON ((-117.85000 33.56667, -117.80833 33.5..."
4,1999-01-01,0.0,33.8,0.00538,137.3,49.0,1.9,293.2,279.9,29.0,13.3,13.6,43.0,1.9,0.80,"POLYGON ((-117.80833 33.56667, -117.76667 33.5..."


In [7]:
perimeters = gpd.read_file('../../data/geoMAC/sd_fire_final_dedupe.shp')
perimeters['date'] = pd.to_datetime(perimeters.final_date, format="%Y-%m-%d")
perimeters.drop(perimeters.columns.difference(['date', 'geometry', 'acres', 'fire_name']), axis=1, inplace=True)
perimeters.crs = {'init' :'epsg:3857'}
perimeters = perimeters.to_crs(epsg=4326)
print(perimeters.shape)
perimeters.head()

C:\Users\Mike\AppData\Local\Continuum\anaconda3\envs\geo\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


(557, 4)


,fire_name,acres,geometry,date
0,67,43.394566,"POLYGON ((-116.95196 32.93964, -116.95149 32.9...",2006-08-11
1,67,43.691695,"POLYGON ((-116.95196 32.93964, -116.95149 32.9...",2006-08-11
2,76-2,11.910000,"POLYGON ((-117.22833 33.28174, -117.22824 33.2...",2010-06-23
3,76-2,11.912337,"POLYGON ((-117.22967 33.27916, -117.22970 33.2...",2010-06-23
4,78 #4,99.974220,"POLYGON ((-116.93929 33.08838, -116.93909 33.0...",2007-07-09


In [8]:
%%time
fire_occurred = gpd.sjoin(df[['date', 'geometry']], perimeters)
fire_occurred = fire_occurred[fire_occurred.date_left == fire_occurred.date_right]
fire_occurred

C:\Users\Mike\AppData\Local\Continuum\anaconda3\envs\geo\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)


Wall time: 31min 38s


,date_left,geometry,index_right,fire_name,acres,date_right
2598868,2004-02-11,"POLYGON ((-117.80833 33.56667, -117.76667 33.5...",220,EL MORO,4.767303,2004-02-11
1770628,2002-06-26,"POLYGON ((-117.80833 33.56667, -117.76667 33.5...",289,LAGUNA,83.299934,2002-06-26
1770629,2002-06-26,"POLYGON ((-117.76667 33.56667, -117.72500 33.5...",289,LAGUNA,83.299934,2002-06-26
1770676,2002-06-26,"POLYGON ((-117.80833 33.52500, -117.76667 33.5...",289,LAGUNA,83.299934,2002-06-26
1770677,2002-06-26,"POLYGON ((-117.76667 33.52500, -117.72500 33.5...",289,LAGUNA,83.299934,2002-06-26
...,...,...,...,...,...,...
4031066,2006-12-05,"POLYGON ((-116.89167 32.52500, -116.85000 32.5...",70,BORDER 30,63.240620,2006-12-05
4031066,2006-12-05,"POLYGON ((-116.89167 32.52500, -116.85000 32.5...",71,BORDER 30,63.665359,2006-12-05
6801146,2012-05-17,"POLYGON ((-116.89167 32.52500, -116.85000 32.5...",73,BORDER 6,11.372368,2012-05-17
6830378,2012-06-07,"POLYGON ((-116.89167 32.52500, -116.85000 32.5...",133,Border 6,11.448854,2012-06-07


In [9]:
fire_occurred.fire_name.value_counts()

Witch         557
Harris        209
Poomacha      182
CEDAR          96
Border 3       88
             ... 
EL MORO         1
HARBISON        1
BORDER #34      1
BONITA          1
Border 12       1
Name: fire_name, Length: 315, dtype: int64

In [10]:
df['fire_occurred'] = False
df.loc[fire_occurred.index, 'fire_occurred'] = True
df.loc[fire_occurred.index, 'acres_burned'] = fire_occurred.acres
df.loc[fire_occurred.index, 'fire_name'] = fire_occurred.fire_name
df.head()

,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry,fire_occurred,acres_burned,fire_name
0,1999-01-01,0.0,40.3,0.00589,138.0,123.0,1.6,293.1,281.1,24.0,16.0,15.5,34.0,1.7,0.74,"POLYGON ((-117.97500 33.56667, -117.93333 33.5...",False,NaN,NaN
1,1999-01-01,0.0,39.8,0.00590,137.2,123.0,1.6,293.1,281.2,24.0,16.0,15.5,35.0,1.7,0.74,"POLYGON ((-117.93333 33.56667, -117.89167 33.5...",False,NaN,NaN
2,1999-01-01,0.0,38.2,0.00580,137.2,123.0,1.7,293.2,281.2,26.0,15.2,14.9,37.0,1.7,0.77,"POLYGON ((-117.89167 33.56667, -117.85000 33.5...",False,NaN,NaN
3,1999-01-01,0.0,36.4,0.00567,137.3,49.0,1.8,293.3,280.3,27.0,15.0,14.8,38.0,1.8,0.76,"POLYGON ((-117.85000 33.56667, -117.80833 33.5...",False,NaN,NaN
4,1999-01-01,0.0,33.8,0.00538,137.3,49.0,1.9,293.2,279.9,29.0,13.3,13.6,43.0,1.9,0.80,"POLYGON ((-117.80833 33.56667, -117.76667 33.5...",False,NaN,NaN


In [11]:
df.head()

,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry,fire_occurred,acres_burned,fire_name
0,1999-01-01,0.0,40.3,0.00589,138.0,123.0,1.6,293.1,281.1,24.0,16.0,15.5,34.0,1.7,0.74,"POLYGON ((-117.97500 33.56667, -117.93333 33.5...",False,NaN,NaN
1,1999-01-01,0.0,39.8,0.00590,137.2,123.0,1.6,293.1,281.2,24.0,16.0,15.5,35.0,1.7,0.74,"POLYGON ((-117.93333 33.56667, -117.89167 33.5...",False,NaN,NaN
2,1999-01-01,0.0,38.2,0.00580,137.2,123.0,1.7,293.2,281.2,26.0,15.2,14.9,37.0,1.7,0.77,"POLYGON ((-117.89167 33.56667, -117.85000 33.5...",False,NaN,NaN
3,1999-01-01,0.0,36.4,0.00567,137.3,49.0,1.8,293.3,280.3,27.0,15.0,14.8,38.0,1.8,0.76,"POLYGON ((-117.85000 33.56667, -117.80833 33.5...",False,NaN,NaN
4,1999-01-01,0.0,33.8,0.00538,137.3,49.0,1.9,293.2,279.9,29.0,13.3,13.6,43.0,1.9,0.80,"POLYGON ((-117.80833 33.56667, -117.76667 33.5...",False,NaN,NaN


In [12]:
df.fire_occurred.value_counts()

False    6824133
True        2167
Name: fire_occurred, dtype: int64

In [13]:
tmp = df.geometry.apply(lambda poly: poly.exterior.coords[0])
df['longitude'] = tmp.apply(lambda p: p[0])
df['latitude'] = tmp.apply(lambda p: p[1])
df.drop('geometry', axis=1).head()

,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,fire_occurred,acres_burned,fire_name,longitude,latitude
0,1999-01-01,0.0,40.3,0.00589,138.0,123.0,1.6,293.1,281.1,24.0,16.0,15.5,34.0,1.7,0.74,False,NaN,NaN,-117.975000,33.566667
1,1999-01-01,0.0,39.8,0.00590,137.2,123.0,1.6,293.1,281.2,24.0,16.0,15.5,35.0,1.7,0.74,False,NaN,NaN,-117.933333,33.566667
2,1999-01-01,0.0,38.2,0.00580,137.2,123.0,1.7,293.2,281.2,26.0,15.2,14.9,37.0,1.7,0.77,False,NaN,NaN,-117.891667,33.566667
3,1999-01-01,0.0,36.4,0.00567,137.3,49.0,1.8,293.3,280.3,27.0,15.0,14.8,38.0,1.8,0.76,False,NaN,NaN,-117.850000,33.566667
4,1999-01-01,0.0,33.8,0.00538,137.3,49.0,1.9,293.2,279.9,29.0,13.3,13.6,43.0,1.9,0.80,False,NaN,NaN,-117.808333,33.566667


In [14]:
df.drop('geometry', axis=1).to_parquet("integratedData.parquet.gz", compression="gzip")

# Uploading to s3

In [15]:
s3_url = "s3://dse-cohort5-group5/wildfire_capstone/integratedData.parquet.gz"
df.drop('geometry', axis=1).to_parquet(s3_url, compression="gzip")

In [16]:
import boto3
s3_url = 'dse-cohort5-group5'
s3 = boto3.client("s3")
all_objects = s3.list_objects(Bucket=s3_url)
all_objects

{'ResponseMetadata': {'RequestId': 'EF90F9405B1915FC',
  'HostId': '5jGryo7JHg02wFuhikJWpgPRmT4SOV1JAsKKeV/UtglZaEVZN0AYrKBFk9wqyfH6y7PC7Nw9SVo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '5jGryo7JHg02wFuhikJWpgPRmT4SOV1JAsKKeV/UtglZaEVZN0AYrKBFk9wqyfH6y7PC7Nw9SVo=',
   'x-amz-request-id': 'EF90F9405B1915FC',
   'date': 'Sun, 10 May 2020 01:05:47 GMT',
   'x-amz-bucket-region': 'us-west-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'IsTruncated': True,
 'Marker': '',
 'Contents': [{'Key': 'test/',
   'LastModified': datetime.datetime(2020, 2, 14, 2, 26, 50, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'kcoakley+cohort5group5',
    'ID': 'fe3a1755292b139ad5397ba537aa0aff32ef6ebba07fe45dcbe486d81e1ccc11'}},
  {'Key': 'wildfire_capstone/GeomacHashes.parquet',
   'LastModified': datetime.datetime(2020,